# 6 Visualise labelled data and remove artefacts

Code that let us overlay each frame of video with outputs from the models. And create time series plots. 

In [ ]:
import os
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import ultralytics.utils as ultrautils
import utils
import display
import json

In [ ]:
videos_in = r"..\LookitLaughter.test"
metadata_file = "_LookitLaughter.xlsx"
data_out = r"..\data\1_interim"
videos_out = r"..\data\2_final"

#a couple of files for testing
VIDEO_FILE  = os.path.join(videos_in, "2UWdXP.joke1.rep2.take1.Peekaboo.mp4")
VIDEO_FILE2 = os.path.join(videos_in, "2UWdXP.joke2.rep1.take1.NomNomNom.mp4")
AUDIO_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.wav")
AUDIO_FILE2 = os.path.join(data_out, "2UWdXP.joke2.rep1.take1.NomNomNom.wav")
SPEECH_FILE = os.path.join(data_out, "2UWdXP.joke1.rep2.take1.Peekaboo.json")
SPEECH_FILE2 = os.path.join(data_out, "2UWdXP.joke2.rep1.take1.NomNomNom.json")

testset = [VIDEO_FILE, VIDEO_FILE2] 

In [ ]:
processedvideos = utils.getprocessedvideos(data_out)
processedvideos.head()

## 6.1 Add annotations onto a video.



In [ ]:
#let's grab a single frame from the video

framenum = 60
video = cv2.VideoCapture(VIDEO_FILE)
video.set(cv2.CAP_PROP_POS_FRAMES, framenum)
ret, frame = video.read()
video.release()

if ret:
    plt.imshow(frame)
    plt.show()



videoname = os.path.basename(VIDEO_FILE)

kpts = utils.getKeyPoints(processedvideos,videoname)
kpts.head()


In [ ]:
#let's grab a single frame from the video
bboxlabels, bboxes, xycs = utils.getframekpts(kpts, framenum)
print(bboxlabels)
print(bboxes)
print(xycs)

In [ ]:
h, w = frame.shape[:2]
print(h,w)    

frame = display.drawOneFrame(frame, bboxlabels, bboxes, xycs,"Peek-a-boo")

plt.imshow(frame)
plt.show()


In [ ]:
#check captions for this video at 
with open(SPEECH_FILE) as f:
    speechjson = json.load(f)
caption = display.WhisperExtractCurrentCaption(speechjson, framenum, 15)
print(caption)

In [ ]:


#let's get all the annotations for this video
kpts = utils.getKeyPoints(processedvideos,videoname)
facedata = utils.getFaceData(processedvideos,videoname)
speechdata = utils.getSpeechData(processedvideos,videoname)


In this demo, the extract movement algorithm has mislabelled the adult and the child (the labels get applied at random). We need to swap the labels around.

In [ ]:
kpts = utils.relabelPersonIndex(kpts, person = "child", index = 0, newPerson = "temp", newIndex = 100)
kpts = utils.relabelPersonIndex(kpts, person = "adult", index = 1, newPerson = "child", newIndex = 0)
kpts = utils.relabelPersonIndex(kpts, person = "temp", index = 100,newPerson = "adult",  newIndex = 1)   

In [ ]:

annotatedVideo = display.createAnnotatedVideo(VIDEO_FILE, kpts, facedata, speechdata, videos_out, True)

In [ ]:
#Add the audio file back onto the annotated video
#annotatedVideo = r"..\data\2_final\2UWdXP.joke1.rep2.take1.Peekaboo_annotated.mp4"

from moviepy.editor import *
videoclip = VideoFileClip(annotatedVideo)
audioclip = AudioFileClip(AUDIO_FILE)

videoclip = videoclip.set_audio(audioclip)


In [ ]:
audioclip.ipython_display()

In [ ]:
videoclip.ipython_display()


In [ ]:
#note need to save with a different name as the original video is still open
videoclip.write_videofile(r"..\data\2_final\2UWdXP.joke1.rep2.take1.Peekaboo_annotated_audio.mp4")

## 6.2 TODO Add annotations to all vidoes.

Generate annotated videos for all videos in the test set.

In [ ]:

for index, r in processedvideos.iterrows():

    videoname = os.path.basename(r["VideoID"])
    try: 
        #let's get all the annotations for this video
        kpts = utils.getKeyPoints(processedvideos,videoname)
        facedata = utils.getFaceData(processedvideos,videoname)
        speechdata = utils.getSpeechData(processedvideos,videoname)
    except FileNotFoundError:
        print(f"Data error for {videoname}\n" + )
        continue
    

        print(f"Already processed {r['VideoID']}")
utils.saveprocessedvideos(processedvideos, data_out)

# 6.3 TODO Correct person labels in all videos.

Swap parent and child if these are wrong. Ignore other people in video.